# Yelp API - Lab


## Introduction 

We've seen how the Yelp API works and how to create basic visualizations using Folium. It's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 

* Using pagination and multiple functions, gather large amounts of data from an API, parse the data and make sense of it with meaningful analysis
* Create maps using Folium

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you but make the request below.

In [1]:
# Import libraries and API key
import requests
import json
import time
import pandas as pd
from ipyleaflet import Map, basemaps, basemap_to_tiles, Marker, Polyline

def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys("/Users/tree/.secret/yelp_api.json")

api_key = keys['api_key']

## Pagination

Now that you have an initial response, you can examine the contents of the JSON container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [4]:
##This is the API Call Portion, builds a DataFrame of all obtined data

# This first portion is an initial get request that creates the dataframe, it then goes into a while loop and
# appends onto the original dataframe with more data. Yelp API limits it to 1000 returns in total.
import requests
term = 'Pizza'
location = 'Brooklyn NY'
SEARCH_LIMIT = 50
offset = 0
url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
counter = 0
url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset
            }

initial_response = requests.get(url, headers=headers, params=url_params)
businesses = pd.DataFrame.from_dict(initial_response.json()['businesses'])

while counter < 400:
# while len(businesses) < initial_response.json()['total']:
    offset += 50
    url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset
            }
    response = requests.get(url, headers=headers, params=url_params)
    if response.status_code == 200:
        bus50 = pd.DataFrame.from_dict(response.json()['businesses'])
        businesses = pd.concat([businesses, bus50], ignore_index = True)
        time.sleep(1)
    else:
        break
    counter += 1
    


In [3]:
len(businesses)

1000

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [5]:
# Your code here
df = businesses

In [15]:
df

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,f-and-f-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.67732, 'longitude': -73.99817}",,5679.579337,aTXpZudGAKCihlF_zaqBIg,https://s3-media3.fl.yelpcdn.com/bphoto/S8hZ6h...,False,"{'address1': '459 Court St', 'address2': '', '...",F & F Pizzeria,,NaN,3.5,39,[],https://www.yelp.com/biz/f-and-f-pizzeria-broo...
1,julianas-pizza-brooklyn-5,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7026153030093, 'longitude': -7...",(718) 596-6700,7214.079409,ysqgdbSrezXgVwER2kQWKA,https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTx...,False,"{'address1': '19 Old Fulton St', 'address2': '...",Juliana's Pizza,+17185966700,$$,4.5,2106,[],https://www.yelp.com/biz/julianas-pizza-brookl...
2,l-industrie-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71162, 'longitude': -73.95783}",(718) 599-0002,6771.011722,v1DHGRNCH9247WLYoaoA9A,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,"{'address1': '254 S 2nd St', 'address2': '', '...",L'industrie Pizzeria,+17185990002,$,5.0,488,"[delivery, pickup]",https://www.yelp.com/biz/l-industrie-pizzeria-...
3,lucali-brooklyn-3,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.6818, 'longitude': -74.00024}",(718) 858-4086,6096.250479,Q9F2ocrmYuGt1yn3M7MOBw,https://s3-media3.fl.yelpcdn.com/bphoto/Yl_eZ_...,False,"{'address1': '575 Henry St', 'address2': None,...",Lucali,+17188584086,$$,4.5,1453,[],https://www.yelp.com/biz/lucali-brooklyn-3?adj...
4,di-fara-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.625093, 'longitude': -73.961531}",(718) 258-1367,3559.746344,C8j0q4Ma_S5hBGuAI-aaww,https://s3-media3.fl.yelpcdn.com/bphoto/4dfe9q...,False,"{'address1': '1424 Ave J', 'address2': '', 'ad...",Di Fara Pizza,+17182581367,$$,4.0,3362,"[delivery, pickup]",https://www.yelp.com/biz/di-fara-pizza-brookly...
5,barboncino-pizza-and-bar-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.67206, 'longitude': -73.95717}",(718) 483-8834,2663.491653,xRiLLXjeM2wOvVTYJH2__A,https://s3-media3.fl.yelpcdn.com/bphoto/48doX4...,False,"{'address1': '781 Franklin Ave', 'address2': '...",Barboncino Pizza & Bar,+17184838834,$$,4.0,921,[],https://www.yelp.com/biz/barboncino-pizza-and-...
6,joe-and-sals-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.6694466400113, 'longitude': -7...",(718) 484-8732,2555.936856,pQo-KWQxuH26y0IzZbQj0A,https://s3-media4.fl.yelpcdn.com/bphoto/V4YjC0...,False,"{'address1': '842 Franklin Ave', 'address2': '...",Joe & Sal's Pizza,+17184848732,$,4.0,109,"[delivery, pickup]",https://www.yelp.com/biz/joe-and-sals-pizza-br...
7,robertas-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.705043534338, 'longitude': -73...",(718) 417-1118,5881.757582,FhXjAc6nKLf414KxYujUHw,https://s3-media4.fl.yelpcdn.com/bphoto/8fY9fA...,False,"{'address1': '261 Moore St', 'address2': '', '...",Roberta's,+17184171118,$$,4.0,2752,"[delivery, pickup]",https://www.yelp.com/biz/robertas-brooklyn-2?a...
8,lombardos-of-bay-ridge-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.634741, 'longitude': -74.0284259}",(718) 238-7100,7659.838650,qoBcWhGwlcUsKIQ2nB22vA,https://s3-media2.fl.yelpcdn.com/bphoto/dApR-o...,False,"{'address1': '279 71st St', 'address2': None, ...",Lombardo's of Bay Ridge,+17182387100,$$,5.0,110,[pickup],https://www.yelp.com/biz/lombardos-of-bay-ridg...
9,patsys-pizzeria-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.6818695, 'longitude': -73.976181}",(718) 622-2268,4520.468918,xrgzB0pvsxeGEIboE6I2gg,https://s3-media4.fl.yelpcdn.com/bphoto/1n5xL5...,False,"{'address1': '450 Dean St', 'address2': '', 'a...",Patsy's Pizzeria,+17186222268,$$,4.5,501,"[delivery, pickup]",https://www.yelp.com/biz/patsys-pizzeria-brook...


In [6]:
keys = df.coordinates.iloc[0].keys()
keys

dict_keys(['latitude', 'longitude'])

In [7]:
## splits out coordinates column into two columns
new_cols = []
for key in keys:
    new_col = 'coordinates_{}'.format(key) #Create new column name
    df[new_col] = df.coordinates.map(lambda x: x[key]) #Create a new column
    new_cols.append(new_col)
df[new_cols].head()
df

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url,coordinates_latitude,coordinates_longitude
0,f-and-f-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.67732, 'longitude': -73.99817}",,5679.579337,aTXpZudGAKCihlF_zaqBIg,https://s3-media3.fl.yelpcdn.com/bphoto/S8hZ6h...,False,"{'address1': '459 Court St', 'address2': '', '...",F & F Pizzeria,,NaN,3.5,39,[],https://www.yelp.com/biz/f-and-f-pizzeria-broo...,40.677320,-73.998170
1,julianas-pizza-brooklyn-5,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.7026153030093, 'longitude': -7...",(718) 596-6700,7214.079409,ysqgdbSrezXgVwER2kQWKA,https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTx...,False,"{'address1': '19 Old Fulton St', 'address2': '...",Juliana's Pizza,+17185966700,$$,4.5,2106,[],https://www.yelp.com/biz/julianas-pizza-brookl...,40.702615,-73.993416
2,l-industrie-pizzeria-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.71162, 'longitude': -73.95783}",(718) 599-0002,6771.011722,v1DHGRNCH9247WLYoaoA9A,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,"{'address1': '254 S 2nd St', 'address2': '', '...",L'industrie Pizzeria,+17185990002,$,5.0,488,"[pickup, delivery]",https://www.yelp.com/biz/l-industrie-pizzeria-...,40.711620,-73.957830
3,di-fara-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.625093, 'longitude': -73.961531}",(718) 258-1367,3559.746344,C8j0q4Ma_S5hBGuAI-aaww,https://s3-media3.fl.yelpcdn.com/bphoto/4dfe9q...,False,"{'address1': '1424 Ave J', 'address2': '', 'ad...",Di Fara Pizza,+17182581367,$$,4.0,3362,"[pickup, delivery]",https://www.yelp.com/biz/di-fara-pizza-brookly...,40.625093,-73.961531
4,lucali-brooklyn-3,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.6818, 'longitude': -74.00024}",(718) 858-4086,6096.250479,Q9F2ocrmYuGt1yn3M7MOBw,https://s3-media3.fl.yelpcdn.com/bphoto/Yl_eZ_...,False,"{'address1': '575 Henry St', 'address2': None,...",Lucali,+17188584086,$$,4.5,1453,[],https://www.yelp.com/biz/lucali-brooklyn-3?adj...,40.681800,-74.000240
5,robertas-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.705043534338, 'longitude': -73...",(718) 417-1118,5881.757582,FhXjAc6nKLf414KxYujUHw,https://s3-media4.fl.yelpcdn.com/bphoto/8fY9fA...,False,"{'address1': '261 Moore St', 'address2': '', '...",Roberta's,+17184171118,$$,4.0,2752,"[pickup, delivery]",https://www.yelp.com/biz/robertas-brooklyn-2?a...,40.705044,-73.933571
6,barboncino-pizza-and-bar-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.67206, 'longitude': -73.95717}",(718) 483-8834,2663.491653,xRiLLXjeM2wOvVTYJH2__A,https://s3-media3.fl.yelpcdn.com/bphoto/48doX4...,False,"{'address1': '781 Franklin Ave', 'address2': '...",Barboncino Pizza & Bar,+17184838834,$$,4.0,921,[],https://www.yelp.com/biz/barboncino-pizza-and-...,40.672060,-73.957170
7,joe-and-sals-pizza-brooklyn,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.6694466400113, 'longitude': -7...",(718) 484-8732,2555.936856,pQo-KWQxuH26y0IzZbQj0A,https://s3-media4.fl.yelpcdn.com/bphoto/V4YjC0...,False,"{'address1': '842 Franklin Ave', 'address2': '...",Joe & Sal's Pizza,+17184848732,$,4.0,109,"[pickup, delivery]",https://www.yelp.com/biz/joe-and-sals-pizza-br...,40.669447,-73.959587
8,lombardos-of-bay-ridge-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}]","{'latitude': 40.634741, 'longitude': -74.0284259}",(718) 238-7100,7659.838650,qoBcWhGwlcUsKIQ2nB22vA,https://s3-media2.fl.yelpcdn.com/bphoto/dApR-o...,False,"{'address1': '279 71st St', 'address2': None, ...",Lombardo's of Bay Ridge,+17182387100,$$,5.0,110,[pickup],https://www.yelp.com/biz/lombardos-of-bay-ridg...,40.634741,-74.028426
9,patsys-pizzeria-brooklyn-2,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...","{'latitude': 40.6818695, 'longitude': -73.976181}",(718) 622-2268,4520.468918,xrgzB0pvsxeGEIboE6I2gg,https://s3-med

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [99]:
## takes locational data and puts into a list with the name, I would like to add rating and price later
lat_list = []
for lat in df['coordinates_latitude']:
    lat_list.append(lat)
    
lon_list = []
for long in df['coordinates_longitude']:
    lon_list.append(long)
    
## make a list of restaurant names
name_list = []
for name in df['name']:
    name_list.append(name)
    
rating_list = []
for rating in df['rating']:
    rating_list.append(rating)
    
price_list = []
for price in df['price']:
    price_list.append(price)

In [89]:
lat_lon_name = list(zip(lat_list, lon_list, name_list, rating_list, price_list))

In [104]:
# this was a subset I used for testing, and then realized putting in all 1000 markers is A LOT so I kept using it
lat_lon_subset = lat_lon_name[0:100]



In [100]:
# Your code here

m = Map(
    layers=(basemap_to_tiles(basemaps.Esri.WorldStreetMap, "2017-04-08"), ),
    center=(40.6782, -73.9442),
    zoom=11
)

for mark in mark_list_subset:
    marker = Marker(location=(mark[0], mark[1]), draggable=False, title=mark[2])
    m.add_layer(marker)

In [101]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [103]:
## Next steps would be to clean the listed restaurants to be ONLY from Brooklyn, as well as filter out places that are
# not exclusively a pizza joint. 

## Summary

Nice work! In this lab, you've made multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice interactive map to display the results using Folium! Well done!